In [22]:
import pandas as pd
import sys
try:
    import folium
except:
    !{sys.executable} -m pip install folium
    import folium
from folium import plugins
from folium.plugins import MarkerCluster
try:
    import altair as alt
except:
    !{sys.executable} -m pip install altair
    import altair as alt

In [2]:
airbnb_data = pd.read_csv('airbnb/hawaii_listings.csv')
airbnb_data['count'] = 1
airbnb_data

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,count
0,5065,MAUKA BB,7257,Wayne,Hawaii,Hamakua,20.042660,-155.432590,Entire home/apt,85,2,42,2020-03-22,0.41,1,250,1
1,5269,Upcountry Hospitality in the 'Auwai Suite,7620,Lea & Pat,Hawaii,South Kohala,20.027400,-155.702000,Entire home/apt,150,3,12,2021-04-08,0.10,3,188,1
2,5387,Hale Koa Studio & 1 Bedroom Units!!,7878,Edward,Hawaii,South Kona,19.430810,-155.880690,Entire home/apt,85,5,176,2021-06-08,1.66,3,176,1
3,5389,Keauhou Villa,7878,Edward,Hawaii,North Kona,19.566040,-155.961990,Entire home/apt,239,6,31,2021-06-26,0.57,3,153,1
4,5390,STAY AT PRINCE KUHIO!,7887,Todd,Kauai,Koloa-Poipu,21.881510,-159.473460,Entire home/apt,96,3,150,2021-06-17,2.07,1,131,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21803,50898372,1 Bedroom Ocean Front Condo in Honokowai - Sle...,118578746,Chase 'N Rainbows,Maui,Lahaina,20.953171,-156.685957,Entire home/apt,265,5,0,NaN,NaN,137,228,1
21804,50898551,1 Bedroom Ocean Front Condo in Honokowai - Sle...,118578746,Chase 'N Rainbows,Maui,Lahaina,20.952492,-156.687250,Entire home/apt,367,5,0,NaN,NaN,137,218,1
21805,50900058,Hawaii Life Rentals Presents Island Sands 110 ...,680107,Kahea,Maui,Kihei-Makena,20.795679,-156.506651,Entire home/apt,189,3,0,NaN,NaN,137,226,1
21806,50900602,Maui Resort Rentals: Honua Kai – Incredible 6 ...,39073224,Maui Resort Rentals,Maui,Lahaina,20.943433,-156.690599,Entire home/apt,3691,5,0,NaN,NaN,261,64,1


In [3]:
# map = folium.Map(location=[37, -121], zoom_start=3)

# mc = MarkerCluster()

# for index, row in geolocation_df.iterrows():
#     mc.add_child(folium.Marker(location=[row['latitude'], row['longitude']]))
    
# stationArr = geolocation_df[['latitude', 'longitude']].as_matrix()

# map.add_child(mc)


In [4]:
august_df = pd.read_csv('airbnb/august_calendar.csv')
august_df

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,87671,8/6/2021,t,$227.00,$227.00,7.0,1125.0
1,87671,8/7/2021,t,$227.00,$227.00,7.0,1125.0
2,87671,8/8/2021,t,$227.00,$227.00,7.0,1125.0
3,87671,8/9/2021,t,$227.00,$227.00,7.0,1125.0
4,87671,8/17/2021,t,$227.00,$227.00,7.0,1125.0
...,...,...,...,...,...,...,...
207792,50897989,8/27/2021,t,$139.00,$139.00,1.0,365.0
207793,50897989,8/28/2021,t,$139.00,$139.00,1.0,365.0
207794,50897989,8/29/2021,t,$139.00,$139.00,1.0,365.0
207795,50897989,8/30/2021,t,$129.00,$129.00,1.0,365.0


In [5]:
listing_with_nights = august_df.groupby(['listing_id']).sum().reset_index(drop=False)[['listing_id', 'maximum_nights']]
listing_with_nights

,listing_id,maximum_nights
0,5269,3200.0
1,5387,6750.0
2,5389,6750.0
3,5434,1460.0
4,5930,360.0
...,...,...
17939,50898372,280.0
17940,50898551,308.0
17941,50900058,455.0
17942,50900602,360.0


In [6]:
merged_data = airbnb_data.merge(listing_with_nights, left_on='id', right_on='listing_id', how='right')
merged_data.drop(columns='listing_id', inplace=True)
merged_data

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,count,maximum_nights
0,5269,Upcountry Hospitality in the 'Auwai Suite,7620,Lea & Pat,Hawaii,South Kohala,20.027400,-155.702000,Entire home/apt,150,3,12,2021-04-08,0.10,3,188,1,3200.0
1,5387,Hale Koa Studio & 1 Bedroom Units!!,7878,Edward,Hawaii,South Kona,19.430810,-155.880690,Entire home/apt,85,5,176,2021-06-08,1.66,3,176,1,6750.0
2,5389,Keauhou Villa,7878,Edward,Hawaii,North Kona,19.566040,-155.961990,Entire home/apt,239,6,31,2021-06-26,0.57,3,153,1,6750.0
3,5434,Kauai Ocean View 3 Story Tiki Tower Princeville,7984,Ahlea,Kauai,North Shore Kauai,22.220800,-159.469890,Entire home/apt,375,2,119,2021-06-23,1.60,2,217,1,1460.0
4,5930,Poipu Beach Guest House - Orchid - SurfSong,10768,John,Kauai,Koloa-Poipu,21.881390,-159.472480,Private room,140,3,185,2021-06-29,2.42,4,103,1,360.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17939,50898372,1 Bedroom Ocean Front Condo in Honokowai - Sle...,118578746,Chase 'N Rainbows,Maui,Lahaina,20.953171,-156.685957,Entire home/apt,265,5,0,NaN,NaN,137,228,1,280.0
17940,50898551,1 Bedroom Ocean Front Condo in Honokowai - Sle...,118578746,Chase 'N Rainbows,Maui,Lahaina,20.952492,-156.687250,Entire home/apt,367,5,0,NaN,NaN,137,218,1,308.0
17941,50900058,Hawaii Life Rentals Presents Island Sands 110 ...,680107,Kahea,Maui,Kihei-Makena,20.795679,-156.506651,Entire home/apt,189,3,0,NaN,NaN,137,226,1,455.0
17942,50900602,Maui Resort Rentals: Honua Kai – Incredible 6 ...,39073224,Maui Resort Rentals,Maui,Lahaina,20.943433,-156.690599,Entire home/apt,3691,5,0,NaN,NaN,261,64,1,360.0


In [7]:
usapoi_data = pd.read_csv('poi_factory/poi_hawaii_data.csv')
usapoi_data

,longitude,latitude,name,info
0,-159.492460,21.885750,Allerton National Botanical Garden,NaN
1,-159.475740,22.212730,Hanalei Valley Lookout,NaN
2,-159.562670,21.919870,Hanapepe Lookout,NaN
3,-159.335350,22.042570,Hikinaakala Heiau,NaN
4,-159.645670,22.150340,Kalalau Lookout,NaN
...,...,...,...,...
92,-155.881076,19.418295,South Kona Fruit Stand,84-4770 Mamalahoa Hwy. Captain Cook HI 808/328...
93,-155.995168,19.643350,Kona Tacos,75-5595 Palani Rd. Kailua-Kona HI 808/329-9049
94,-159.670837,21.958202,Daniel Thiebaut,65-1259 Kawaihae Rd. Waimea HI 808/887-2200
95,-155.070245,19.725163,Nihon Restaurant,123 Lihiwai St. Hilo HI 808/969-1133


In [23]:
map = folium.Map(location=[37, -121], zoom_start=3)

stationArr = airbnb_data[['latitude', 'longitude', 'count']].values


#Commented out to save space. Make sure your computer can handle folium charts!
map.add_children(plugins.HeatMap(stationArr, radius=15))

mc = MarkerCluster()

for index, row in usapoi_data.iterrows():
    mc.add_child(folium.Marker(location=[row['latitude'], row['longitude']]))
    
map.add_child(mc)

#Illustrating clusters (count of individual places rows) with heatmap
map

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  import sys


https://kepler.gl/demo

In [20]:
august_cal = pd.read_csv('airbnb/august_calendar.csv')
august_cal['date'] = pd.to_datetime(august_cal['date']).dt.strftime('%Y/%m/%d %s')
august_cal

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,87671,2021-08-06 00:00:00,t,$227.00,$227.00,7.0,1125.0
1,87671,2021-08-07 00:00:00,t,$227.00,$227.00,7.0,1125.0
2,87671,2021-08-08 00:00:00,t,$227.00,$227.00,7.0,1125.0
3,87671,2021-08-09 00:00:00,t,$227.00,$227.00,7.0,1125.0
4,87671,2021-08-17 00:00:00,t,$227.00,$227.00,7.0,1125.0
...,...,...,...,...,...,...,...
207792,50897989,2021-08-27 00:00:00,t,$139.00,$139.00,1.0,365.0
207793,50897989,2021-08-28 00:00:00,t,$139.00,$139.00,1.0,365.0
207794,50897989,2021-08-29 00:00:00,t,$139.00,$139.00,1.0,365.0
207795,50897989,2021-08-30 00:00:00,t,$129.00,$129.00,1.0,365.0
